# A full business solution


### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [6]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage



In [7]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')

if api_key :
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gemini-1.5-pro'
chatModel=ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=api_key, temperature=0.4)


API key looks good so far


In [8]:
for chunk in chatModel.stream("give me short story in b1 profienty?"):
    print(chunk.content, end='', flush=True)

The aroma of woodsmoke clung to Amelia's thick wool coat as she trudged through the freshly fallen snow.  The forest path, usually a vibrant green tunnel, was now a hushed corridor of white.  Amelia shivered, not entirely from the cold.  She’d been avoiding this walk for weeks, ever since the argument.  The argument that had left a silence thicker than the snow, a silence that stretched between her and her grandfather, Elias.

Elias, a renowned woodcarver, lived a solitary life in a small cabin nestled deep within the woods.  He’d taught Amelia everything she knew about the forest, about respecting its rhythms and its secrets.  But their shared passion had fractured.  He’d disapproved of her decision to leave for the city, to pursue her art in a bustling, impersonal world.  His words, sharp and unexpected, had stung.  "You're abandoning the very thing that nourishes you," he'd said.

Now, with the holidays approaching, the guilt gnawed at her.  She clutched the small, intricately carve

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links,
    using Selenium to fetch fully rendered content.
    """

    def __init__(self, url, driver_path="C:\webdriver\chromedriver-win64\chromedriver.exe", headless=True):
        self.url = url

        options = Options()
        if headless:
            options.add_argument("--headless")
            options.add_argument("--no-sandbox")
            options.add_argument("--disable-dev-shm-usage")

        service = Service(driver_path)
        self.driver = webdriver.Chrome(service=service, options=options)

        self.driver.get(url)

        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        html = self.driver.page_source
        self.driver.quit()

        soup = BeautifulSoup(html, 'html.parser')

        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [35]:
ed = Website("https://huggingface.co")
print(ed.get_contents())

Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
google/gemma-3n-E4B-it-litert-preview
Updated
1 day ago
•
248
multimodalart/isometric-skeumorphic-3d-bnb
Updated
7 days ago
•
1.24k
•
240
Wan-AI/Wan2.1-VACE-14B
Updated
3 days ago
•
12.9k
•
257
nvidia/parakeet-tdt-0.6b-v2
Updated
6 days ago
•
374k
•
1.02k
ByteDance-Seed/BAGEL-7B-MoT
Updated
about 2 hours ago
•
198
Browse 1M+ models
Spaces
Running
6.92k
6.92k
DeepSite
🐳
Generate any application with DeepSeek
Running
on
Zero
337
337
LTX Video Fast
🎥
ultra-fast video model, LTX 0.9.7 13B distilled
Running
530
530
FLUX Pro Unlimited
🔥
Use the FLUX-Pro model as much as you want.
Running
on
Zero
449
449
DreamO
🐨
A Unified

In [36]:
ed = Website("https://huggingface.co")
ed.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/multimodalart/isometric-skeumorphic-3d-bnb',
 '/Wan-AI/Wan2.1-VACE-14B',
 '/nvidia/parakeet-tdt-0.6b-v2',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/ByteDance/DreamO',
 '/spaces/smolagents/computer-agent',
 '/spaces',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/PrimeIntellect/INTELLECT-2-RL-Dataset',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly

## First step: Have Gemini-1.5-pro figure out which links are relevant

### Use a call to Gemini-1.5-pro  to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!



In [37]:
link_system_prompt = """
You are a helpful assistant that filters relevant links from a webpage.
Your goal is to identify and return links that would be useful in a company brochure,
such as links to: social media links, About, Careers, Company Info, Blog, or Contact pages.

You MUST respond with a valid JSON object in the format:

{
  "links": [
    {
      "type": "about page",
      "url": "https://example.com/about"
    },
    {
      "type": "careers page",
      "url": "https://example.com/careers"
    }
  ]
}

Only include all links that are relevant to a brochure.
Do NOT include links to Terms of Service, Privacy Policy, or Email links.
Only return the JSON. Do not add explanations or extra text.
"""


In [39]:
print(link_system_prompt)


You are a helpful assistant that filters relevant links from a webpage.
Your goal is to identify and return links that would be useful in a company brochure,
such as links to: social media links, About, Careers, Company Info, Blog, or Contact pages.

You MUST respond with a valid JSON object in the format:

{
  "links": [
    {
      "type": "about page",
      "url": "https://example.com/about"
    },
    {
      "type": "careers page",
      "url": "https://example.com/careers"
    }
  ]
}

Only include all links that are relevant to a brochure.
Do NOT include links to Terms of Service, Privacy Policy, or Email links.
Only return the JSON. Do not add explanations or extra text.



In [40]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [41]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://huggingface.co - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/google/gemma-3n-E4B-it-litert-preview
/multimodalart/isometric-skeumorphic-3d-bnb
/Wan-AI/Wan2.1-VACE-14B
/nvidia/parakeet-tdt-0.6b-v2
/ByteDance-Seed/BAGEL-7B-MoT
/models
/spaces/enzostvs/deepsite
/spaces/Lightricks/ltx-video-distilled
/spaces/NihalGazi/FLUX-Pro-Unlimited
/spaces/ByteDance/DreamO
/spaces/smolagents/computer-agent
/spaces
/datasets/openbmb/Ultra-FineWeb
/datasets/disco-eth/EuroSpeech
/datasets/nvidia/OpenCodeReasoning
/datasets/PrimeIntellect/INTELLECT-2-RL-Dataset
/datasets/nvidia/OpenMathReasoning
/datasets
/join
/pricing#endpoints
/pricing#spaces
/pricing
/enterprise
/enterprise
/enterprise
/

In [42]:
import re


def get_links(url):
    website = Website(url)
    
    messages = [
        SystemMessage(content=link_system_prompt),
        HumanMessage(content=get_links_user_prompt(website))
    ]
    
    response = chatModel.invoke(messages)
    if response.content.strip().startswith("```json"):
        response.content = re.sub(r"```json\s*", "", response.content)
        response.content = re.sub(r"```$", "", response.content.strip())

    
    try:
        return json.loads(response.content)
    except Exception:
        raise ValueError(f"LLM response was not valid JSON: {response.content}")

In [43]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'github', 'url': 'https://github.com/huggingface'},
  {'type': 'twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'discord', 'url': 'https://huggingface.co/join/discord'}]}

In [44]:


huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/multimodalart/isometric-skeumorphic-3d-bnb',
 '/Wan-AI/Wan2.1-VACE-14B',
 '/nvidia/parakeet-tdt-0.6b-v2',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/ByteDance/DreamO',
 '/spaces/smolagents/computer-agent',
 '/spaces',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/PrimeIntellect/INTELLECT-2-RL-Dataset',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly

## Second step: make the brochure!



In [46]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [47]:
print(get_all_details("https://huggingface.co")[3956:])

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discord', 'url': 'https://huggingface.co/join/discord'}]}
Twitter
LinkedIn
Discord
Zhihu
WeChat



about page
Webpage Title:
Brand assets - Hugging Face
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
Hugging Face · Brand assets
HF Logos
.svg
.png
.ai
.svg
.png
.ai
.svg
.png
.ai
HF Colors
#FFD21E
#FF9D00
#6B7280
HF Bio
Hugging Face is the collaboration platform for the machine learning community.

The Hugging Face Hub works as a central pla

In [48]:
system_prompt = """
You are an assistant that creates a concise brochure about a company using ONLY the exact information provided from relevant pages of the company website. 
Do NOT add any opinions, assumptions, or information not explicitly present in the text.
Present the output clearly in markdown format, including sections about company overview, services, industries served, career opportunities (if explicitly mentioned), and contact information.
If a particular detail (e.g. careers or company culture) is not present in the provided information, do NOT mention it or speculate.
"""


In [49]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    return user_prompt

In [51]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discord', 'url': 'https://huggingface.co/join/discord'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ngoogle/gemma-3n-E4B-it-litert-preview\nUpdated\n1 day ago\n•\n248\nmultimodalart/isometric-skeumorphic-3d-bnb\nUpdated\n7 days ago\n•\n1.24k\n•\n240\nWan-AI/Wan2.1-VACE-14B\nUpdated\n3 days ago\n•\n12.9k\n•\n257\nnvidia/parakeet-tdt-0.6b-v2\nUpdated\n6 days ago\n•\n374k\n•\n1.02k\nByteDance-Seed/BAGEL-7B-MoT\nUpdated\nabout 2 hours ago\n•\n199\nBrowse 1M+ models\nSpaces\nRunning\n6.92k\n6.

In [52]:
def create_brochure(company_name, url):
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=get_brochure_user_prompt(company_name, url))
    ]
    response = chatModel.invoke(messages)
    result = response.content
    display(Markdown(result))


In [53]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discord', 'url': 'https://huggingface.co/join/discord'}]}


## Hugging Face: The AI community building the future.

**Company Overview**

Hugging Face is the platform where the machine learning community collaborates on models, datasets, and applications.  Their mission is to democratize good machine learning.

**Services**

* **Models:** Host and collaborate on unlimited public models. Explore text, image, video, audio, and 3D modalities.
* **Datasets:** Host and collaborate on unlimited public datasets.
* **Spaces:** Host and collaborate on unlimited public applications.
* **Compute:** Deploy on optimized Inference Endpoints or update Spaces applications to a GPU. Starting at $0.60/hour for GPU.
* **Enterprise:** Enterprise-grade security, access controls, and dedicated support. Starting at $20/user/month. Includes Single Sign-On, Regions, Priority Support, Audit Logs, Resource Groups, and Private Datasets Viewer.

**Industries Served**

Hugging Face is used by over 50,000 organizations, including:

* Ai2 (Enterprise non-profit)
* AI at Meta (Enterprise company)
* Amazon (company)
* Google (company)
* Intel (company)
* Microsoft (company)
* Grammarly (Enterprise company)
* Writer (Enterprise company)


**Open Source Projects**

Hugging Face contributes to numerous open-source projects, including:

* Transformers: State-of-the-art ML for PyTorch, TensorFlow, JAX (144,629 stars on GitHub)
* Diffusers: State-of-the-art Diffusion models in PyTorch (29,056 stars on GitHub)
* Safetensors: Safe way to store/distribute neural network weights (3,273 stars on GitHub)
* Hub Python Library: Python client to interact with the Hugging Face Hub (2,611 stars on GitHub)
* Tokenizers: Fast tokenizers optimized for research & production (9,694 stars on GitHub)
* TRL: Train transformers LMs with reinforcement learning (13,839 stars on GitHub)
* Transformers.js: State-of-the-art ML running directly in your browser (13,636 stars on GitHub)
* smolagents: Smol library to build great agents in Python (18,946 stars on GitHub)
* PEFT: Parameter-efficient finetuning for large language models (18,476 stars on GitHub)
* Datasets: Access & share datasets for any ML tasks (20,156 stars on GitHub)
* Text Generation Inference: Serve language models with TGI optimized toolkit (10,145 stars on GitHub)
* Accelerate: Train PyTorch models with multi-GPU, TPU, mixed precision (8,734 stars on GitHub)


**Contact Information**

* **Website:** huggingface.co
* **For press inquiries:** Contact the team through the contact form on the about page.
* **Social Media:** GitHub, Twitter, LinkedIn, Discord, Zhihu, WeChat (Links available on huggingface.co)

In [ ]:
create_brochure("Deka Technology", "https://dekatechs.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://dekatechs.com'}, {'type': 'about page', 'url': 'https://dekatechs.com/about-us/'}, {'type': 'case studies', 'url': 'https://dekatechs.com/case-studies/'}, {'type': 'blog', 'url': 'https://dekatechs.com/blog/'}, {'type': 'contact', 'url': 'https://dekatechs.com/contact/'}, {'type': 'careers', 'url': 'https://dekatechs.com/careers/'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/dekatechs/mycompany/verification/'}, {'type': 'twitter', 'url': 'https://twitter.com/dekatechnology'}, {'type': 'facebook', 'url': 'https://www.facebook.com/dekatechs/'}]}


# Deka Technology Brochure

## Company Overview

Deka Technology has pioneered IT services in Istanbul for 7 years, consistently delivering business value with the latest technology. Founded in Istanbul in 2017, Deka Technology serves companies globally.  Their mission is to deliver innovative and valuable solutions for digital transformation nationally and internationally.  They operate globally and are proud to serve customers and partners around the world.

Deka Technology's Core Values:

* **Passionate:**  Driven to achieve the best and deliver exceptional results.
* **Respectful:** Treat customers, partners, and team members with respect and appreciation.
* **Ownership:** Take responsibility for tasks and act like owners.
* **Unified:** Create unity in actions and communication to pursue a common goal.
* **Experienced:** Team members with a minimum of a decade's expertise.

## Services

Deka Technology offers a range of IT solutions, including:

* **Managed Services:** Day-to-day support, management, and monitoring of IT.
* **UI/ UX Web Design:** Technology implemented, managed, and monitored for growth.
* **Infrastructure Management:** Planning, design, implementation, maintenance, and development of systems.
* **Software Development:** Planning, design, development, integration, testing, management, and evolution of software solutions.
* **Database Consulting:** Expertise and delivery flexibility from trusted DBA consultants.
* **Data Analytics and Business Intelligence:** State-of-the-art business intelligence solutions for data-driven insights.
* **IT Consulting & Advisory**
* **Web Development**
* **Mobile Development**
* **Cloud Management**
* **Digital Transformation**
* **Robotic Process Automation (RPA)**


## Industries Served

* Consumer Goods
* Transportation & Logistics
* Healthcare
* Banks & Insurance
* Consulting Providers
* Automotive

## Career Opportunities

Deka Technology is growing and expanding its international network. They offer a family atmosphere, agile teams, and opportunities to shape change.  Benefits include:

* Competitive Compensation
* Flexible Working Hours
* Professional Development
* Modern Work Environment
* Diversity and Inclusion
* Teamwork
* Social Responsibility
* Hybrid and Remote Work Opportunities


## Contact Information

* **Phone:** +90 2166066957
* **Email:** info(at)dekatechs.com
* **Headquarters:** Aydınevler Mah. Durak Sok. Ofis in Maltepe A Blk. D: 16 Maltepe – Istanbul
* **R&D Office:** Kırıkkale Üniversitesi Kampüsü Yenişehir Mah. Farabi Cad. 1 Ar-Ge Binası Kat:1 No: 30/44 71450 Yahşihan/Kırıkkale
* **On-site R&D:** Just Work, Meydan İstanbul AVM, Balkan Cd. No:62, 34770 Ümraniye/İstanbul
* **LinkedIn:** [LinkedIn](The provided link is a general LinkedIn login page, not the company's specific page.)
* **Twitter:** [Twitter](The provided link is a general Twitter page, not the company's specific page.)
* **Facebook:** [Facebook](The provided link is a general Facebook login, not the company's specific page.)